# Parte 1: Web Scraping con BeautifulSoup

Utilizando la biblioteca BeautifulSoup en Python, extrae información de la siguiente. Debes extraer la siguiente información:

- El nombre de las rutas que aparecen en la página web.
- Donde esta ubicada la ruta.
- El tipo de ruta. Si esta información añadir "Desconocido".
- Duración de la ruta.
- Los kilometros de la ruta.
- La dificultad de la ruta.
- El esfuerzo de la ruta.
- La descripción de la ruta.
- Tendrás que obtener un DataFrame similar al que observas a continuación

In [ ]:
# antes de empezar importamos las librerías que vamos a usar. 
# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) # para poder visualizar todas las columnas de los DataFrames

In [ ]:
#primero hago el request de la url
url_montañas='https://turismoactiva.com/rutas-de-montana-sierra-de-gredos/'
resp_montañas=requests.get(url_montañas)
#A continuación hago la sopa para poder ir trasteando en los datos
sopa_montañas=BeautifulSoup(resp_montañas.content,'html.parser')

In [ ]:
#Creo una lista vacía donde acabaré metiendo las tuplas con la info de cada ruta
lista_todos_datos=[]
#Creo una iteración para sacar cada uno de los datos de la parte de la web que me interesa,
# valores anteriores de 7 no me interesan y sólo me interesan los impares
for i in range (7,40,2):
    #Creo una lista de datos vacía donde meteré los datos limpios de la sopa
    lista_datos=[]
    #Creo una lista de datos sopa vacía, que será donde meta el texto de los datos que me interesa
    lista_datos_sopa=[]
    #Saco los datos de cada una de las rutas
    datos=sopa_montañas.find_all('div', {'class':'elementor-widget-wrap elementor-element-populated'})[i]
    #Elimino los \xa0
    datos=datos.text.replace('\xa0',' ')
    #Elimino los saltos de línea y creo una lista
    datos=datos.split('\n')
    #Comienzo a iterar en la lista datos, donde se encuentran mis datos de cada ruta y varias posiciones de lista vacía
    for dato in datos:
        #Si la longitud de dato es mayor que 0, es decir, no está vacío se meterá en la lista_datos_sopa, sino pasa.
        #Otra opción habría sido hacer datos.remove('')
        if len(dato)>0:
            lista_datos_sopa.append(dato)
            
        else:
            pass
    #Añadimos toda la lista de datos sopa en la lista de datos, para hacer una lista de listas
    lista_datos.append(lista_datos_sopa)
    #Creo una lista de datos final donde meter los datos 
    lista_datos_final=[]
    #Itero en la lista de datos
    for montaña in lista_datos:
        #Hago un try except para evitar errores
        try:
            #Si el tipo de ruta no esta en la posición 2 de la lista montaña
            if 'Tipo de ruta:' not in montaña[2]:
                
                for k in range(0,6):
                    #Itero en la lista montaña
                    info=montaña[k]
                    #Intento partir el texto de mi lista en el ': ' y meto en datos final la posición 1
                    try:
                        lista_datos_final.append(info.split(': ')[1])
                    #si no puedo hacer eso meto la info directamente
                    except:
                        lista_datos_final.append(info)
                #Busco la descripción montaña en el último lugar
                descripción=montaña[-1]
                lista_datos_final.append(descripción)
                #Inserto en la posición 2, la que sería de tipo de dato, el valor de Desconocido
                lista_datos_final.insert(2,'Desconocido')
                #Añado en la lista de todos los datos la tupla de lisa de datos final 
                lista_todos_datos.append(tuple(lista_datos_final))
            else:
                #repito el código pero si sí que hubiera tipo de ruta
                for k in range(0,7):
                    info=montaña[k]
                    try:
                        lista_datos_final.append(info.split(': ')[1])
                        
                    except:
                        lista_datos_final.append(info)
                descripción=montaña[-1]
                lista_datos_final.append(descripción)
                lista_todos_datos.append(tuple(lista_datos_final))
        except:
            pass
             
        

    


In [ ]:
#Creo una lista con los títulos de las columnas
lista_columnas=['nombre', 'donde','tipo','duracion','km','dificultad','esfuerzo','descripcion']

In [ ]:
#Creo el dataframe
df_montañas=pd.DataFrame(lista_todos_datos)
df_montañas.columns=lista_columnas
df_montañas
#Hay 2 que la parte de km, dificultad y esfuerzo están desordenados

# Parte 2: Obtención de Datos Climatológicos con la API de AEMET

Utiliza la API de AEMET para obtener información climatológica de la Sierra de Gredos. En concreto deberás usar el endpoint de "predicciones-especificas" la predicción de montaña para la sierra de gredos. Debe realizar las siguientes tareas:

- Incluir la temperatura máxima en la Sierra de Gredos.
- Incluir la temperatura mínima en la Sierra de Gredos.
- Incluir la fecha en la que se recopilaron los datos.
- Incluir la sierra de donde vienen los datos.